In [1]:
import os
import shutil
import errno 
from PIL import Image

In [2]:
source_folder = os.path.join('..', 'data', '_brut')

def delete_folder(folder_path):
    """Deletes a folder and its contents.

    Args:
        folder_path (str): The path to the folder to delete.
    """

    if not os.path.exists(folder_path):
        print(f"Folder '{folder_path}' does not exist.")
        return

    try:
        os.rmdir(folder_path)
        print(f"Folder '{folder_path}' deleted successfully.")
    except OSError as e:
        if e.errno == errno.ENOTEMPTY:
            print(f"Error deleting folder '{folder_path}': Folder is not empty.")
        else:
            print(f"Error deleting folder '{folder_path}': {e}")

def create_directories(base_path):
    os.makedirs(os.path.join(base_path, 'images', 'train'), exist_ok=True)
    os.makedirs(os.path.join(base_path, 'images', 'val'), exist_ok=True)
    os.makedirs(os.path.join(base_path, 'labels', 'train'), exist_ok=True)
    os.makedirs(os.path.join(base_path, 'labels', 'val'), exist_ok=True)

def copy_files(src_paths, dest_base_path, subfolder):
    for src in src_paths:
        src_file = src
        relative_path = os.path.relpath(src_file, start=source_folder)
        dest_file = os.path.join(dest_base_path, subfolder)
        dest_folder = os.path.dirname(dest_file)
        os.makedirs(dest_folder, exist_ok=True)
        try:
            shutil.copy(src_file, dest_file)
        except Exception as e:
            print(f"Error copying {src_file}: {e}")

# Dataset YOLO Total

In [3]:
fruit_list = sorted(os.listdir(source_folder))
fruit_to_idx = {fruit : i for i, fruit in enumerate(fruit_list)}


# Création du dictionnaire de données
dataset = {
            'images': {
                'train': [],
                'val': []
            },
            'labels': {
                'train': [],
                'val': []
            }
        }

# Remplissage du dictionnaire de données
for fruit in fruit_list:
    images_path = os.path.join(source_folder, fruit)
    labels_path = os.path.join(images_path, 'Label')

    images = sorted([f for f in os.listdir(images_path) if f.endswith('.jpg')])[:400]
    labels = sorted([f for f in os.listdir(labels_path) if f.endswith('.txt')])[:400]

    if len(images) != len(labels):
        raise ValueError(f"Le nombre d'images et de labels ne correspond pas dans le dossier '{fruit}'")

    split_point = int(len(images) * 0.8)

    dataset['images']['train'].extend([os.path.join(images_path, img) for img in images[:split_point]])
    dataset['images']['val'].extend([os.path.join(images_path, img) for img in images[split_point:]])

    dataset['labels']['train'].extend([os.path.join(labels_path, lbl) for lbl in labels[:split_point]])
    dataset['labels']['val'].extend([os.path.join(labels_path, lbl) for lbl in labels[split_point:]])



# path sortie
dataset_path = os.path.join('..', 'data', 'yolo_total', 'datasets')

# Delete existing directories if necessary
delete_folder(os.path.join(dataset_path, 'images'))
delete_folder(os.path.join(dataset_path, 'labels'))

# Create the dataset structure
create_directories(dataset_path)

# Copy files
for split in ['train', 'val']:
    copy_files(dataset['images'][split], dataset_path, os.path.join('images', split))
    copy_files(dataset['labels'][split], dataset_path, os.path.join('labels', split))

Folder '../data/yolo_total/datasets/images' does not exist.
Folder '../data/yolo_total/datasets/labels' does not exist.


Encodage des labels

In [4]:
def openImage(filename):
    try:
        img = Image.open( os.path.join('..', 'data', 'yolo_total', 'datasets', 'images', 'train', filename.split(".")[0] + '.jpg' ))
        return img
    except:
        pass  

    try:
        img = Image.open( os.path.join('..', 'data', 'yolo_total', 'datasets', 'images', 'val', filename.split(".")[0] + '.jpg' ))
        return img
    except:
        pass  


def modify_txt_files(directory, fruit_to_id):
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            with open(filepath, "r") as f:
                lines = f.readlines()  
  
            new_lines = []
            for line in lines:
                values = line.strip().split()
                if len(values) == 6:
                    # Cas particulier pour "Bell pepper"
                    class_name = "Bell pepper"
                    _, _,x_tl, y_tl, x_br, y_br = values
                else:
                    class_name, x_tl, y_tl, x_br, y_br = values
                
                class_id = fruit_to_id.get(class_name, -1)  # Get ID, default -1 if not found
                
                if class_id != -1:  # Only modify if class is found
                    img = openImage( filename.split(".")[0] + '.jpg' )
                    img_w, img_h = img.size

                    x_br = float(x_br)
                    x_tl = float(x_tl)
                    y_br = float(y_br)
                    y_tl = float(y_tl)

                    x_center = (x_br + x_tl)/2/img_w
                    y_center = (y_tl + y_br)/2/img_h

                    w_bb = (x_br - x_tl)/img_w
                    h_bb = (y_br - y_tl)/img_h

                    new_line = f"{class_id} {x_center} {y_center} {w_bb} {h_bb}\n"
                    new_lines.append(new_line)
                

            with open(filepath, "w") as f:
                f.writelines(new_lines)

# Replace "your_directory" with the actual path to your directory
modify_txt_files(os.path.join("..", "data", "yolo_total", "datasets","labels", "train"), fruit_to_idx)
modify_txt_files(os.path.join("..", "data", "yolo_total", "datasets","labels", "val"), fruit_to_idx)


# Dataset YOLO Segmentation

In [5]:
def supprimer_premier_mot(fichier):
    """
    Supprime le premier mot de chaque ligne d'un fichier .txt.

    Args:
        fichier (str): Chemin vers le fichier .txt.
    """

    with open(fichier, 'r') as f:
        lignes = f.readlines()

    with open(fichier, 'w') as f:
        for ligne in lignes:
            mots = ligne.split()
            if mots:
                f.write(' '.join(mots[1:]) + '\n')

def copier_et_modifier(source, destination):
    """
    Copie un dossier et modifie les fichiers .txt.

    Args:
        source (str): Chemin du dossier source.
        destination (str): Chemin du dossier destination.
    """

    for root, dirs, files in os.walk(source):
        for dir in dirs:
            src_dir = os.path.join(root, dir)
            dest_dir = os.path.join(destination, os.path.relpath(src_dir, source))
            if dir not in ['images', 'datasets']:  # Ne pas copier les dossiers images et datasets
                shutil.copytree(src_dir, dest_dir, dirs_exist_ok=True)
        for file in files:
            if file.endswith('.txt'):
                src_file = os.path.join(root, file)
                dest_file = os.path.join(destination, os.path.relpath(src_file, source))
                os.makedirs(os.path.dirname(dest_file), exist_ok=True)
                shutil.copy2(src_file, dest_file)
                supprimer_premier_mot(dest_file)
            else:
                # Copier les autres fichiers sans modification
                src_file = os.path.join(root, file)
                dest_file = os.path.join(destination, os.path.relpath(src_file, source))
                os.makedirs(os.path.dirname(dest_file), exist_ok=True)
                shutil.copy2(src_file, dest_file)

# Chemins des dossiers
source_folder = os.path.join("..", "data", "yolo_total", "datasets")
destination_folder = os.path.join("..", "data", "yolo_segmentation", "datasets")

# Copier et modifier
copier_et_modifier(source_folder, destination_folder)